### Import library

In [1]:
import os
os.chdir('../../')
os.getcwd()

'C:\\Users\\csia7\\OneDrive\\문서\\GitHub\\WQBrain_2024_API'

In [2]:
import ace_lib as ace
import helpful_functions as hf
import pandas as pd
import requests
import plotly.express as px
import pygwalker as pyg
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

### Start session
Enter credentials once - they will be saved to local folder and loaded each time

In [3]:
s = ace.start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_KXq52HHwrVQu2XTXmHN4yxskx1AS

 


## Global region Alpha Template

In [4]:
expression_template = f'''
group = (country+1)*group_max(pv13_52_minvol_1m_all_delay_1_sector, market) + pv13_52_minvol_1m_all_delay_1_sector;
group_neutralize(ts_scale(group_backfill(fnd23_intfvalld1_ecns, industry, 252, std=1), 252), densify(group))
'''

#### Step 4. Apply generate_alpha function to the expression list
In generate alpha function you can specify region, universe, decay, delay and other simulation settings

이제 만들어둔 expressions 을 사용해 alpha simulation 을 해줍니다.

In [61]:
#?ace.generate_alpha

In [5]:
#when you send multiple alphas for simulation, please make sure all alphas of a single list should have common settings
#alphas with different settings should be sent in a different list, for instance below list has all alphas with same settings

alpha_list = [ace.generate_alpha(expression_template , region= "GLB", universe = "MINVOL1M", neutralization = 'COUNTRY', truncation = 0.01, delay = 1, decay = x) for x in range(5)]
alpha_list[0]

{'type': 'REGULAR',
 'settings': {'instrumentType': 'EQUITY',
  'region': 'GLB',
  'universe': 'MINVOL1M',
  'delay': 1,
  'decay': 0,
  'neutralization': 'COUNTRY',
  'truncation': 0.01,
  'pasteurization': 'ON',
  'testPeriod': 'P0Y0M0D',
  'unitHandling': 'VERIFY',
  'nanHandling': 'OFF',
  'language': 'FASTEXPR',
  'visualization': False},
 'regular': '\ngroup = (country+1)*group_max(pv13_52_minvol_1m_all_delay_1_sector, market) + pv13_52_minvol_1m_all_delay_1_sector;\ngroup_neutralize(ts_scale(group_backfill(fnd23_intfvalld1_ecns, industry, 252, std=1), 252), densify(group))\n'}

### Simulate alpha list, get simulation result

simulate_alpha_list_multi will do a multi-simulation if list of alphas is greater than 10, which is the case here

the returned object will contain simulation results for all alphas as a list

In [6]:
#alpha expressions are sliced to first 10 for demonstration purpose

result = ace.simulate_alpha_list_multi(s, alpha_list)

Warning, list of alphas too short, single concurrent simulations will be used instead of multisimulations


100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [11:46<00:00, 141.21s/it]


위 코드를 실행하면 시뮬레이션이 시작됩니다. 100개의 알파에 대략 1시간 정도 소요되니, 인터넷 연결이 끊기지 않게 주의하시고, 너무 많은 알파를 한 번에 돌리기보다는 몇개씩 끊어서 돌려도 좋습니다.

In [7]:
#prettify_result function can be used from the helpful_functions library to take a look at IS stats of all the simulated alphas

result_st1 = hf.prettify_result(result, detailed_tests_view=False)
result_st1

,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,start_date,alpha_id,expression,concentrated_weight,high_turnover,is_ladder_sharpe,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover
0,3622137,20000000,4074,3938,0.0266,0.0350,0.0260,0.002626,1.21,2.29,2012-01-22,EwjlvM9,\ngroup = (country+1)*group_max(pv13_52_minvol...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
1,3621977,20000000,4074,3938,0.0266,0.0350,0.0260,0.002626,1.21,2.29,2012-01-22,AwplN3X,\ngroup = (country+1)*group_max(pv13_52_minvol...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
2,3577008,20000000,4074,3937,0.0260,0.0345,0.0264,0.002656,1.19,2.26,2012-01-22,ad07bXR,\ngroup = (country+1)*group_max(pv13_52_minvol...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
3,3559172,20000000,4074,3937,0.0257,0.0344,0.0267,0.002676,1.18,2.24,2012-01-22,Xp379Yx,\ngroup = (country+1)*group_max(pv13_52_minvol...,PASS,PASS,PASS,PASS,PASS,PASS,PASS
4,3545713,20000000,4074,3936,0.0254,0.0342,0.0271,0.002694,1.17,2.23,2012-01-22,bLmj6Aq,\ngroup = (country+1)*group_max(pv13_52_minvol...,PASS,PASS,PASS,PASS,PASS,PASS,PASS


prettify 함수를 이용해 dataframe 형식으로 변환하면 결과를 보기 더 편합니다.

In [8]:
result_st1.to_csv('2024_09_03_GLB.csv')

### How to submit?

Create a list of submittable alphas - alphas that have no FAIL in is_tests

시뮬레이션을 마친 알파 중 제출 가능한 알파들을 제출해 봅시다.

In [9]:
#to take a look at the combined result of all new alphas

is_tests_df = hf.concat_is_tests(result)
is_tests_df

,alpha_id,endDate,limit,message,name,result,startDate,themes,value,year
0,EwjlvM9,NaN,1.58,NaN,LOW_SHARPE,PASS,NaN,NaN,2.2900,NaN
1,EwjlvM9,NaN,1.00,NaN,LOW_FITNESS,PASS,NaN,NaN,1.2100,NaN
2,EwjlvM9,NaN,0.01,NaN,LOW_TURNOVER,PASS,NaN,NaN,0.0266,NaN
3,EwjlvM9,NaN,0.70,NaN,HIGH_TURNOVER,PASS,NaN,NaN,0.0266,NaN
4,EwjlvM9,NaN,NaN,NaN,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
60,bLmj6Aq,NaN,NaN,NaN,DATA_DIVERSITY,PENDING,NaN,NaN,NaN,NaN
61,bLmj6Aq,NaN,NaN,NaN,PROD_CORRELATION,PENDING,NaN,NaN,NaN,NaN
62,bLmj6Aq,NaN,NaN,NaN,REGULAR_SUBMISSION,PENDING,NaN,NaN,NaN,NaN
63,bLmj6Aq,2020-01-25,2.02,NaN,IS_LADDER_SHARPE,PASS,2022-01-24,NaN,2.4600,2.0


In [10]:
#making a list of failed alphas
failed_alphas = is_tests_df.query('result=="FAIL"')['alpha_id'].unique()

#making a list of passed alphas
passed_alphas = list(set(is_tests_df['alpha_id']).difference(failed_alphas))

print(f'Failed alphas:{failed_alphas}\nPassed alphas:{passed_alphas}')

Failed alphas:[]
Passed alphas:['Xp379Yx', 'AwplN3X', 'ad07bXR', 'EwjlvM9', 'bLmj6Aq']


위 함수들을 이용해 All Pass (correlation 제외) 가 나온 알파들의 id 를 쉽게 구별해 낼 수 있습니다. 주의하셔야 할 점은 is_test 함수에 넣는 input 값은 prettify 하지 않은, alpha simulation 을 마쳤을 때 출력되는 원본 형태를 입력해야 된다는 겁니다.

In [82]:
ace.get_check_submission(s, passed_alphas[0])

,name,result,limit,value,competitions,alpha_id
0,LOW_SHARPE,PASS,1.25,2.1700,NaN,z92n2P8
1,LOW_FITNESS,PASS,1.00,1.1900,NaN,z92n2P8
2,LOW_TURNOVER,PASS,0.01,0.3861,NaN,z92n2P8
3,HIGH_TURNOVER,PASS,0.70,0.3861,NaN,z92n2P8
4,CONCENTRATED_WEIGHT,PASS,NaN,NaN,NaN,z92n2P8
5,LOW_SUB_UNIVERSE_SHARPE,PASS,0.76,0.9300,NaN,z92n2P8
6,SELF_CORRELATION,PASS,0.70,0.5198,NaN,z92n2P8
7,PROD_CORRELATION,FAIL,0.70,0.7659,NaN,z92n2P8
8,REGULAR_SUBMISSION,PASS,4.00,0.0000,NaN,z92n2P8
9,MATCHES_COMPETITION,PASS,NaN,NaN,"[{'id': 'challenge', 'name': 'Challenge'}]",z92n2P8


위 함수로 각 알파가 correlation 을 포함한 모든 기준에서 통과를 했는지 확인 해 볼 수 있습니다.

In [83]:
for alpha_id in passed_alphas:
    print(ace.get_check_submission(s, alpha_id))

                      name result  limit   value  \
0               LOW_SHARPE   PASS   1.25  2.1700   
1              LOW_FITNESS   PASS   1.00  1.1900   
2             LOW_TURNOVER   PASS   0.01  0.3861   
3            HIGH_TURNOVER   PASS   0.70  0.3861   
4      CONCENTRATED_WEIGHT   PASS    NaN     NaN   
5  LOW_SUB_UNIVERSE_SHARPE   PASS   0.76  0.9300   
6         SELF_CORRELATION   PASS   0.70  0.5198   
7         PROD_CORRELATION   FAIL   0.70  0.7659   
8       REGULAR_SUBMISSION   PASS   4.00  0.0000   
9      MATCHES_COMPETITION   PASS    NaN     NaN   

                                 competitions alpha_id  
0                                         NaN  z92n2P8  
1                                         NaN  z92n2P8  
2                                         NaN  z92n2P8  
3                                         NaN  z92n2P8  
4                                         NaN  z92n2P8  
5                                         NaN  z92n2P8  
6                           

                      name result  limit   value  \
0               LOW_SHARPE   PASS   1.25  2.4700   
1              LOW_FITNESS   PASS   1.00  1.3400   
2             LOW_TURNOVER   PASS   0.01  0.4417   
3            HIGH_TURNOVER   PASS   0.70  0.4417   
4      CONCENTRATED_WEIGHT   PASS    NaN     NaN   
5  LOW_SUB_UNIVERSE_SHARPE   PASS   0.86  1.0800   
6         SELF_CORRELATION   PASS   0.70  0.3222   
7         PROD_CORRELATION   FAIL   0.70  0.7670   
8       REGULAR_SUBMISSION   PASS   4.00  0.0000   
9      MATCHES_COMPETITION   PASS    NaN     NaN   

                                 competitions alpha_id  
0                                         NaN  xxG0den  
1                                         NaN  xxG0den  
2                                         NaN  xxG0den  
3                                         NaN  xxG0den  
4                                         NaN  xxG0den  
5                                         NaN  xxG0den  
6                           

                      name result  limit   value  \
0               LOW_SHARPE   PASS   1.25  2.3300   
1              LOW_FITNESS   PASS   1.00  1.3900   
2             LOW_TURNOVER   PASS   0.01  0.3689   
3            HIGH_TURNOVER   PASS   0.70  0.3689   
4      CONCENTRATED_WEIGHT   PASS    NaN     NaN   
5  LOW_SUB_UNIVERSE_SHARPE   PASS   0.82  1.0300   
6         SELF_CORRELATION   PASS   0.70  0.3848   
7         PROD_CORRELATION   FAIL   0.70  0.8931   
8       REGULAR_SUBMISSION   PASS   4.00  0.0000   
9      MATCHES_COMPETITION   PASS    NaN     NaN   

                                 competitions alpha_id  
0                                         NaN  8zJ1aRV  
1                                         NaN  8zJ1aRV  
2                                         NaN  8zJ1aRV  
3                                         NaN  8zJ1aRV  
4                                         NaN  8zJ1aRV  
5                                         NaN  8zJ1aRV  
6                           

                      name result  limit   value  \
0               LOW_SHARPE   PASS   1.25  2.5800   
1              LOW_FITNESS   PASS   1.00  1.5600   
2             LOW_TURNOVER   PASS   0.01  0.3700   
3            HIGH_TURNOVER   PASS   0.70  0.3700   
4      CONCENTRATED_WEIGHT   PASS    NaN     NaN   
5  LOW_SUB_UNIVERSE_SHARPE   PASS   0.90  1.2600   
6         SELF_CORRELATION   PASS   0.70  0.3681   
7         PROD_CORRELATION   FAIL   0.70  0.8606   
8       REGULAR_SUBMISSION   PASS   4.00  0.0000   
9      MATCHES_COMPETITION   PASS    NaN     NaN   

                                 competitions alpha_id  
0                                         NaN  wRGgxYl  
1                                         NaN  wRGgxYl  
2                                         NaN  wRGgxYl  
3                                         NaN  wRGgxYl  
4                                         NaN  wRGgxYl  
5                                         NaN  wRGgxYl  
6                           

                      name result  limit   value  \
0               LOW_SHARPE   PASS   1.25  2.6100   
1              LOW_FITNESS   PASS   1.00  1.6900   
2             LOW_TURNOVER   PASS   0.01  0.3402   
3            HIGH_TURNOVER   PASS   0.70  0.3402   
4      CONCENTRATED_WEIGHT   PASS    NaN     NaN   
5  LOW_SUB_UNIVERSE_SHARPE   PASS   0.91  1.2500   
6         SELF_CORRELATION   PASS   0.70  0.4079   
7         PROD_CORRELATION   FAIL   0.70  0.8244   
8       REGULAR_SUBMISSION   PASS   4.00  0.0000   
9      MATCHES_COMPETITION   PASS    NaN     NaN   

                                 competitions alpha_id  
0                                         NaN  dx9YMjE  
1                                         NaN  dx9YMjE  
2                                         NaN  dx9YMjE  
3                                         NaN  dx9YMjE  
4                                         NaN  dx9YMjE  
5                                         NaN  dx9YMjE  
6                           

위 함수로 통과한 알파들을 한꺼번에 돌려볼 수 있습니다. 다만 여러개를 돌리면 시간이 꽤 걸리는 점 주의해 주시길 바랍니다.

In [11]:
for alpha_id in passed_alphas:
    hf.set_alpha_properties(s, alpha_id, tags = ['Sep03_GLB'])

위 함수로 통과한 알파들에 태그를 붙일 수 있습니다.

When you got a list of submittable alphas, you can call function submit_alpha()

In [26]:
#calling submit_alpha on all alphas that have passed the submission tests

submit_result = {alpha_id: ace.submit_alpha(s, alpha_id) for alpha_id in passed_alphas}

위 함수로 알파를 제출할 수 있습니다.

알파의 id 로 제출하는 것이기 때문에, 그 알파를 시뮬레이션 한 날짜가 중요합니다. 가령, 1주일 전에 시뮬레이션 해 결과를 저장해 두었던 alpha id로 제출을 한다면 제출 날짜가 1주일 전이 됩니다. 그러니 오늘 제출하고 싶은 alpha 가 있다면 시뮬레이션을 다시 돌려 새로 생긴 alpha id 를 이용해야 합니다. 이 과정은 alpha 에 특정 태그를 달아 Brain 플랫폼에서 직접 하는게 훨씬 수월합니다. (다만, 플랫폼에서는 alpha id 로 알파를 검색할 수 없습니다.)

In [27]:
#submit_result will have return values from the submit_result function

submit_result

{'z92n2P8': False,
 'Jp3oAxE': False,
 'Xpz5Zwm': False,
 'lVGb3QA': False,
 'LpzJJKL': False,
 '5wGAbp1': False,
 'KbzjAvE': False,
 'xxG0den': False,
 '37PZ3vz': False,
 'm879JN9': False,
 'oKGMjGn': False,
 'wRN2mEv': False,
 'qAGqPLK': False,
 'JpzZege': False,
 '8zJ1aRV': False,
 'MpgeWQ6': False,
 'AwzKKxX': False,
 'LpzAMne': False,
 'xxGgPvN': False,
 'z9Agxgo': True,
 'roGvE9j': False,
 'wRGgxYl': False,
 '6w0v3oG': True,
 'EwzXYzR': False,
 'dxzAJ3w': False,
 'Lp8mjaL': False,
 '17GO8GX': False,
 'PpYmYnW': False,
 'dx9YMjE': False,
 'RpzWQ7o': False}

주의하셔야 하실 점은, 하루 최대 알파 제출 한도인 4개를 루프중에 이미 넘기면 그 뒤의 알파들이 제출 가능하더라도 자동으로 제출에 실패하게 됩니다.

### Library Fuctions.

following are some other functions that you can use for your own analysis

**get_alpha_pnl(s, alpha_id)** - to get the pnl for an alpha

**get_alpha_yearly_stats(s, alpha_id)** - to get yearly statistics for an alpha

**get_self_corr(s, alpha_id)** - to get self correlation results for an alpha

**get_prod_corr(s, alpha_id)** - to get prod correlation results for an alpha

**get_check_submission(s, alpha_id)** - to get check submission result for an alpha

**check_self_corr_test(s, alpha_id)** - to check if alpha passes self correlation test (self_corr<0.7)

**check_prod_corr_test(s, alpha_id)** - to check if alpha passes prod correlation test (prod_corr<0.7)

**perfomance_comparison(s, alpha_id)** - to get the result of performance comparison for an alpha merged performance

유용한 추가 함수들입니다. prod correlation 혹은 self correlation 만을 확인하고 싶을 때, 혹은 알파의 pnl 을 확인하고 싶을때 유용히 쓰입니다.